In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("C:/Users/yvonn/OneDrive/Desktop/SUNWAY/202409-S4/Business Intelligence/Assignment/SIandHappiness.csv")

In [3]:
df.head()

,crossid,Country name,year,Nutrition & Basic Medical Care,Water & Sanitation,Shelter,Personal Safety,Access to Basic Knowledge,Access to Information & Communications,Health & Wellness,Environmental Quality,Personal Rights,Personal Freedom & Choice,Inclusiveness,Access to Advanced Education,Life Ladder
0,Afghanistan_2011,Afghanistan,2011,50.349998,49.610001,29.059999,33.910000,27.670000,24.850000,18.209999,31.500000,44.389999,30.150000,30.020000,21.719999,3.832
1,Afghanistan_2012,Afghanistan,2012,52.220001,51.340000,30.410000,33.450001,27.740000,24.340000,20.209999,31.299999,44.389999,30.540001,29.709999,22.230000,3.783
2,Afghanistan_2013,Afghanistan,2013,53.759998,54.209999,32.810001,34.410000,27.860001,30.090000,21.900000,31.080000,44.410000,30.750000,28.480000,22.379999,3.572
3,Afghanistan_2014,Afghanistan,2014,54.759998,56.410000,42.740002,34.840000,27.660000,31.219999,23.090000,30.170000,46.130001,31.320000,27.830000,21.950001,3.131
4,Afghanistan_2015,Afghanistan,2015,55.009998,56.950001,45.740002,36.450001,28.590000,32.130001,23.129999,30.020000,46.180000,32.779999,28.730000,23.280001,3.983


In [4]:
unique_countries = df['Country name'].unique()
len(unique_countries)

230

In [5]:
# Grouping by 'Country name' and calculating the median of 'Life Ladder' for each country
median_life_ladder_by_country = df.groupby('Country name')['Life Ladder'].median()

# Function to fill missing values with the median, but only if there are non-null values in the group
def conditional_fill(x):
    if x.notna().any():  # Check if there are any non-null values in the group
        return x.fillna(x.median())  # Fill missing values with the median
    return x  # If all values are NaN, return the series unchanged

# Applying the conditional imputation
df['Life Ladder'] = df.groupby('Country name')['Life Ladder'].transform(conditional_fill)

In [6]:
# Filter the data to include only rows from 2011 to 2022
df_filtered = df[df['year'].between(2011, 2022)]

# Identify countries where 'Life Ladder' is missing for all years between 2011 and 2022
countries_all_missing = df_filtered.groupby('Country name')['Life Ladder'].apply(lambda x: x.isna().all())

# Get the list of countries where all values are missing
countries_to_remove = countries_all_missing[countries_all_missing].index.tolist()
print(countries_to_remove)


['American Samoa', 'Andorra', 'Antigua and Barbuda', 'Aruba', 'Bahamas', 'Barbados', 'Bermuda', 'British Virgin Islands', 'Brunei Darussalam', 'Cabo Verde', 'Cape Verde', 'Cayman Islands', 'Channel Islands', "Cote d'Ivoire", 'Cuba', 'Curacao', 'Dominica', 'East Asia and Pacific', 'Equatorial Guinea', 'Eritrea', 'Eurasia', 'Europe and North America', 'Faroe Islands', 'Fiji', 'French Polynesia', 'Gibraltar', 'Greenland', 'Grenada', 'Guam', 'Guinea-Bissau', 'Guyana', 'Isle of Man', 'Kiribati', 'Latin America and the Caribbean', 'Liechtenstein', 'Macau', 'Marshall Islands', 'Micronesia, Fed. Sts.', 'Middle East and North Africa', 'Monaco', 'Nauru', 'New Caledonia', 'North Korea', 'Northern Mariana Islands', 'Palau', 'Papua New Guinea', 'Puerto Rico', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Seychelles', 'Sint Maarten (Dutch part)', 'Solomon Islands', 'South Asia', 'St. Kitts and Nevis', 'St. Lucia', 'St. Martin (French part)', 'St. Vincent and the Grenadines', 'Sub-Saharan Africa',

In [7]:
# Remove these countries from the original dataframe
df_cleaned = df_filtered[~df_filtered['Country name'].isin(countries_to_remove)]

# Print the countries that have been removed and their count
print("Countries removed due to all missing values in 'Life Ladder' from 2011 to 2022:")
print(f"Total number of countries removed: {len(countries_to_remove)}")

# Print the total number of countries left in the dataset
unique_countries = set(df_cleaned['Country name'].unique())  # Convert to set
countries_to_remove_set = set(countries_to_remove)  # Convert to set
left_countries = unique_countries - countries_to_remove_set  # Set subtraction to find remaining countries

print(f"Total number of countries left in the dataset: {len(left_countries)}")

Countries removed due to all missing values in 'Life Ladder' from 2011 to 2022:
Total number of countries removed: 69
Total number of countries left in the dataset: 161


In [8]:
# List of independent variable columns
independent_columns = ['Nutrition & Basic Medical Care', 'Water & Sanitation', 'Shelter', 
                       'Personal Safety', 'Access to Basic Knowledge', 'Access to Information & Communications',
                       'Health & Wellness', 'Environmental Quality', 'Personal Rights', 'Personal Freedom & Choice', 
                       'Inclusiveness', 'Access to Advanced Education']

# Filter the dataframe for the years 2011 to 2022
df_cleaned_years = df_cleaned[df_cleaned['year'].between(2011, 2022)]

# Grouping by 'Country name' and checking if each group has continuous missing values in all independent columns
def check_country(group):
    # Check if the group (country) has continuous missing values in each independent variable across the years 2011-2022
    continuous_missing = group[independent_columns].isna().all(axis=1).all()  # All rows must be NaN
    return not continuous_missing  # We want to keep the country if it doesn't have continuous NaN values

# Filter out countries that do not meet the condition
countries_to_keep = df_cleaned_years.groupby('Country name').filter(check_country)

# Identify the countries that were removed by this filtering step
removed_countries = set(df_cleaned['Country name']) - set(countries_to_keep['Country name'])

# Print the countries that have been removed and the count
print("Countries removed due to continuous missing values in all independent variable columns (2011-2022):")
print(removed_countries)
print(f"Total number of countries removed in this step: {len(removed_countries)}")

# df is now filtered to keep only countries that do not have continuous missing values in all independent variable columns
df_saved = df_cleaned_years[df_cleaned_years['Country name'].isin(countries_to_keep['Country name'])].copy()


Countries removed due to continuous missing values in all independent variable columns (2011-2022):
{'Taiwan Province of China', 'Russian Federation', 'Syria', 'Congo (Kinshasa)', 'Iran', 'Vietnam', 'Hong Kong S.A.R. of China', 'North Macedonia', 'Kyrgyz Republic', 'Slovakia', 'Lao PDR', 'South Korea', 'Congo (Brazzaville)'}
Total number of countries removed in this step: 13


In [9]:
# Get unique countries from the filtered dataset
unique_countries_left = df_saved['Country name'].unique()

# Print the total number of unique countries left
print(f"Total number of unique countries left in the dataset: {len(unique_countries_left)}")


Total number of unique countries left in the dataset: 148


In [10]:
# Export the DataFrame to a CSV file
csv_file_path = 'SIandHappinessV2.csv'
df_saved.to_csv(csv_file_path, index=False)

print(f'DataFrame successfully exported to {csv_file_path}')

DataFrame successfully exported to SIandHappinessV2.csv
